The code below is meant for use in Google Colab; using spark to handle parquet files is easier there. To run spark on local machines it would likely require reverting to an older version of Java. Google Colab can run this code without worrring about environment conflicts.

The file used here is `yellow_tripdata-2024-01.parquet`, it contains data on January 2024 Yellow Cab trips in New York City. The file was taken from NYC Taxi and Limousine Commission (TLC)[here](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

It only contains data from one month (January). More data could be combined with this from other months with a union.

NOTE: To run this code in Google Colab, manually upload files from Resources folder. The final product of this code is NYCTaxi.parquet (this data is ready to be train, test, split in spark).

Use Spark to read parquet files. Start by loading in all dependencies.

In [15]:
import os
import pandas as pd
# spark version 3.4.4 performs best; other versions here https://downloads.apache.org/spark/
# spark_version = 'spark-3.4.4'
spark_version = 'spark-3.4.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,619 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,514 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,738 k

In [16]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

For now the file is manually uploaded to memory; `yellow_tripdata_2024-01.parquet` is not cloud based. If this works it will be hosted in an `s3` bucket.

In [ ]:
# Read in data
from pyspark import SparkFiles
data = "/content/yellow_tripdata_2024-01.parquet"
spark.sparkContext.addFile(data)
df = spark.read.parquet(data)
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [ ]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

'PULocationID' and 'DOLocationID' are pick-up and drop-off locations. They use a neighborhood idea as there locations. These IDs are outlined in another smaller table/dataset. Those will need to be used to create a geolocation for those points.

Compare to a clean version of training data used in a kaggle machine learning [competition](https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data) of the same (or similar) data.

In [ ]:
# Training data
training_df = pd.read_csv('train.csv')
training_df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [ ]:
training_df.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

In [ ]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [ ]:
training_df.dtypes

,0
id,object
vendor_id,int64
pickup_datetime,object
dropoff_datetime,object
passenger_count,int64
pickup_longitude,float64
pickup_latitude,float64
dropoff_longitude,float64
dropoff_latitude,float64
store_and_fwd_flag,object


In [ ]:
df.dtypes

[('VendorID', 'int'),
 ('tpep_pickup_datetime', 'timestamp_ntz'),
 ('tpep_dropoff_datetime', 'timestamp_ntz'),
 ('passenger_count', 'bigint'),
 ('trip_distance', 'double'),
 ('RatecodeID', 'bigint'),
 ('store_and_fwd_flag', 'string'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('payment_type', 'bigint'),
 ('fare_amount', 'double'),
 ('extra', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('improvement_surcharge', 'double'),
 ('total_amount', 'double'),
 ('congestion_surcharge', 'double'),
 ('Airport_fee', 'double')]

Found a pdf [here](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf) that outlines what all these column names represent:

`tpep_pickup_datetime`: date/time meter engaged

`tpep_dropff_datetime`: date/time meter disengaged

`Store_andfwd_flag`: Yes or No, record held in memory or not (stored in memory when no server connection)

Other column names are more straightforward. The training data from the [competition](https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data) has far fewer columns, there are no columns to do with fares or money. These could potentially be added to the model during tuning if needed, although for now the raw data will be transformed to look like the training data.

Keep:
vendor_id, pickup time, dropoff time, passenger count, pickup location, dropoff location, and store/flag.

Create:
trip_duration (measure time between start and stop, new column)


In [ ]:
# Create new DataFrame 2024 data
new_training_df = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'PULocationID', 'DOLocationID', 'store_and_fwd_flag']]
new_training_df.show()

+--------------------+---------------------+---------------+------------+------------+------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|PULocationID|DOLocationID|store_and_fwd_flag|
+--------------------+---------------------+---------------+------------+------------+------------------+
| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         186|          79|                 N|
| 2024-01-01 00:03:00|  2024-01-01 00:09:36|              1|         140|         236|                 N|
| 2024-01-01 00:17:06|  2024-01-01 00:35:01|              1|         236|          79|                 N|
| 2024-01-01 00:36:38|  2024-01-01 00:44:56|              1|          79|         211|                 N|
| 2024-01-01 00:46:51|  2024-01-01 00:52:57|              1|         211|         148|                 N|
| 2024-01-01 00:54:08|  2024-01-01 01:26:31|              1|         148|         141|                 N|
| 2024-01-01 00:49:44|  2024-01-01 01:15:47|  

In [ ]:
new_training_df.dtypes

[('tpep_pickup_datetime', 'timestamp_ntz'),
 ('tpep_dropoff_datetime', 'timestamp_ntz'),
 ('passenger_count', 'bigint'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('store_and_fwd_flag', 'string')]

The data types seem to be fine, so let's try to rename columns to make them easier to understand.

In [ ]:
# Rename multiple columns (chain `withColumnRenamed`)
new_training_df = new_training_df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
       .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime") \
       .withColumnRenamed("PULocationID", "pickup_location") \
       .withColumnRenamed("DOLocationID", "dropoff_location") \

new_training_df.show()



+-------------------+-------------------+---------------+---------------+----------------+------------------+
|    pickup_datetime|   dropoff_datetime|passenger_count|pickup_location|dropoff_location|store_and_fwd_flag|
+-------------------+-------------------+---------------+---------------+----------------+------------------+
|2024-01-01 00:57:55|2024-01-01 01:17:43|              1|            186|              79|                 N|
|2024-01-01 00:03:00|2024-01-01 00:09:36|              1|            140|             236|                 N|
|2024-01-01 00:17:06|2024-01-01 00:35:01|              1|            236|              79|                 N|
|2024-01-01 00:36:38|2024-01-01 00:44:56|              1|             79|             211|                 N|
|2024-01-01 00:46:51|2024-01-01 00:52:57|              1|            211|             148|                 N|
|2024-01-01 00:54:08|2024-01-01 01:26:31|              1|            148|             141|                 N|
|2024-01-0

Multiple columns need to be created, all of them relate to pickups and dropoffs. First `trip_duration` needs to be created, then the latitude and longitude of all the locations.

In [ ]:
from pyspark.sql import functions as F

In [ ]:
# Create `trip_duration` from `pickup_datetime` and `dropoff_datetime`
new_training_df = new_training_df.withColumn("pickup_datetime", F.col("pickup_datetime").cast("timestamp"))
new_training_df = new_training_df.withColumn("dropoff_datetime", F.col("dropoff_datetime").cast("timestamp"))

# Calculate the duration in seconds
new_training_df = new_training_df.withColumn("trip_duration",
                   (F.unix_timestamp("dropoff_datetime") - F.unix_timestamp("pickup_datetime")))

# Show the result
new_training_df.select("pickup_datetime", "dropoff_datetime", "trip_duration").show()


+-------------------+-------------------+-------------+
|    pickup_datetime|   dropoff_datetime|trip_duration|
+-------------------+-------------------+-------------+
|2024-01-01 00:57:55|2024-01-01 01:17:43|         1188|
|2024-01-01 00:03:00|2024-01-01 00:09:36|          396|
|2024-01-01 00:17:06|2024-01-01 00:35:01|         1075|
|2024-01-01 00:36:38|2024-01-01 00:44:56|          498|
|2024-01-01 00:46:51|2024-01-01 00:52:57|          366|
|2024-01-01 00:54:08|2024-01-01 01:26:31|         1943|
|2024-01-01 00:49:44|2024-01-01 01:15:47|         1563|
|2024-01-01 00:30:40|2024-01-01 00:58:40|         1680|
|2024-01-01 00:26:01|2024-01-01 00:54:12|         1691|
|2024-01-01 00:28:08|2024-01-01 00:29:16|           68|
|2024-01-01 00:35:22|2024-01-01 00:41:41|          379|
|2024-01-01 00:25:00|2024-01-01 00:34:03|          543|
|2024-01-01 00:35:16|2024-01-01 01:11:52|         2196|
|2024-01-01 00:43:27|2024-01-01 00:47:11|          224|
|2024-01-01 00:51:53|2024-01-01 00:55:43|       

In [ ]:
new_training_df.show()

+-------------------+-------------------+---------------+---------------+----------------+------------------+-------------+
|    pickup_datetime|   dropoff_datetime|passenger_count|pickup_location|dropoff_location|store_and_fwd_flag|trip_duration|
+-------------------+-------------------+---------------+---------------+----------------+------------------+-------------+
|2024-01-01 00:57:55|2024-01-01 01:17:43|              1|            186|              79|                 N|         1188|
|2024-01-01 00:03:00|2024-01-01 00:09:36|              1|            140|             236|                 N|          396|
|2024-01-01 00:17:06|2024-01-01 00:35:01|              1|            236|              79|                 N|         1075|
|2024-01-01 00:36:38|2024-01-01 00:44:56|              1|             79|             211|                 N|          498|
|2024-01-01 00:46:51|2024-01-01 00:52:57|              1|            211|             148|                 N|          366|
|2024-01

The `trip_duration` column is measured in seconds.

Now the hard part. We need to turn the neighborhood codes from before into pickup/dropoff latitude/longitude. Maybe geopandas could handle something like this, or else additional data will be required.

In [ ]:
neighborhood_id_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


There is a shapefile associated with this zoning; it was overlooked earlier and time was lost pursuing GeoJSON files from other sources. Those either did load, or were imprecise with many missing matches for neighborhood names.

In [ ]:
# Install geopandas
!pip install geopandas


In [ ]:
import geopandas as gpd

# Load the shapefile (this automatically handles .shp, .shx, .dbf, etc.)
gdf = gpd.read_file("/content/taxi_zones.shp")

# Display the first few rows of the dataframe to see the attributes
gdf.head()


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.77 256767.698, 1026495.593 2..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.31 144283.336, 936046.565 144..."


In [ ]:
# Load the shapefile
gdf = gpd.read_file("/content/taxi_zones.shp")

# Extract LocationID and centroids (latitude, longitude)
gdf['centroid'] = gdf.geometry.centroid
gdf['latitude'] = gdf['centroid'].y
gdf['longitude'] = gdf['centroid'].x

# Select only LocationID, latitude, and longitude
gdf_location = gdf[['LocationID', 'latitude', 'longitude']]

# Display the first few rows to ensure it looks correct
gdf_location.head()


,LocationID,latitude,longitude
0,1,191376.749531,9.359968e+05
1,2,164018.754403,1.031086e+06
2,3,254265.478659,1.026453e+06
3,4,202959.782391,9.906340e+05
4,5,140681.351376,9.318714e+05


In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName("NeighborhoodMapping").getOrCreate()

# Convert gdf_location pandas DataFrame to Spark DataFrame
gdf_spark = spark.createDataFrame(gdf_location)

# Show the Spark DataFrame to ensure it looks correct
gdf_spark.show()

+----------+------------------+------------------+
|LocationID|          latitude|         longitude|
+----------+------------------+------------------+
|         1|191376.74953083202| 935996.8210162065|
|         2|164018.75440320166|1031085.7186032843|
|         3|254265.47865856893|1026452.6168734727|
|         4| 202959.7823911368| 990633.9806410479|
|         5|140681.35137597343| 931871.3700680139|
|         6| 157998.9356119239|  964319.735448061|
|         7|216719.21816867893|1006496.6791586807|
|         8|222936.08755158543|1005551.5711778702|
|         9| 212969.8490136597|1043002.6774243254|
|        10|186706.49646915717|1042223.6050722591|
|        11| 159429.3168836981| 982170.7832338787|
|        12| 195378.9728508775| 979934.7305677243|
|        13|198691.52543753176|  979792.331028984|
|        14|166921.55042206828| 975952.0350819234|
|        15| 224738.6129755755|1043521.3830455702|
|        16| 217243.7604961802| 1047016.784330188|
|        17| 191215.0683591191|

In [ ]:
# Join new_training_df with gdf_spark on 'pickup_location'
new_training_df_with_pickup = new_training_df.join(
    gdf_spark,
    new_training_df['pickup_location'] == gdf_spark['LocationID'],
    how='left'
).select(
    new_training_df['*'],
    gdf_spark['latitude'].alias('pickup_latitude'),
    gdf_spark['longitude'].alias('pickup_longitude')
)


In [ ]:
# Join new_training_df with gdf_spark on 'dropoff_location'
new_training_df_with_dropoff = new_training_df_with_pickup.join(
    gdf_spark.alias("dropoff_gdf"),  # Alias gdf_spark for the dropoff join
    new_training_df_with_pickup['dropoff_location'] == gdf_spark['LocationID'],
    how='left'
).select(
    new_training_df_with_pickup['*'],
    # Use the alias to specify the dropoff latitude and longitude with col function
    F.col("dropoff_gdf.latitude").alias('dropoff_latitude'),
    F.col("dropoff_gdf.longitude").alias('dropoff_longitude')
)

In [ ]:
new_training_df_with_dropoff.show()

+-------------------+-------------------+---------------+---------------+----------------+------------------+-------------+------------------+------------------+------------------+------------------+
|    pickup_datetime|   dropoff_datetime|passenger_count|pickup_location|dropoff_location|store_and_fwd_flag|trip_duration|   pickup_latitude|  pickup_longitude|  dropoff_latitude| dropoff_longitude|
+-------------------+-------------------+---------------+---------------+----------------+------------------+-------------+------------------+------------------+------------------+------------------+
|2024-01-24 15:49:04|2024-01-24 15:55:15|              1|             43|             237|                 N|          371|224356.21221165065| 993789.0096985319|219305.82782555217| 993769.0237137815|
|2024-01-01 00:43:27|2024-01-01 00:47:11|              2|             68|              90|                 N|          224|211948.91240008172| 984272.7786326221|209708.74184834195| 985089.2186553577|


Now we have a new Spark DataFrame with all the data we need similar to the training data we saw in the competition. Let's rename it so it's easier to understand.


In [ ]:
# Rename to something easy to understand
final_spark_df = new_training_df_with_dropoff
final_spark_df.show()

+-------------------+-------------------+---------------+---------------+----------------+------------------+-------------+------------------+------------------+------------------+------------------+
|    pickup_datetime|   dropoff_datetime|passenger_count|pickup_location|dropoff_location|store_and_fwd_flag|trip_duration|   pickup_latitude|  pickup_longitude|  dropoff_latitude| dropoff_longitude|
+-------------------+-------------------+---------------+---------------+----------------+------------------+-------------+------------------+------------------+------------------+------------------+
|2024-01-24 15:49:04|2024-01-24 15:55:15|              1|             43|             237|                 N|          371|224356.21221165065| 993789.0096985319|219305.82782555217| 993769.0237137815|
|2024-01-01 00:43:27|2024-01-01 00:47:11|              2|             68|              90|                 N|          224|211948.91240008172| 984272.7786326221|209708.74184834195| 985089.2186553577|


In [ ]:
# Export as parquet file
final_spark_df.write.parquet("final_spark_df.parquet")

Now that the final is saved in parquet (as a folder) it can't be downloaded directly from Google Colab. Compress the file first then download it.

In [ ]:
import shutil

# Specify the folder containing the Parquet files
folder_path = '/content/final_spark_df.parquet'  # replace with your folder path

# Create a zip file from the folder
shutil.make_archive('/content/parquet_folder', 'zip', folder_path)


'/content/parquet_folder.zip'

Experiment with reading from `s3` bucket. An issue developped that reading a parquet folder with s3 was not working. Links to a single `.parquet` file do seem to work. Effort here to unzip a compressed `zip` parquet file, but then condense it back into a single `parquet` file.

In [1]:
import shutil

# Find zipped file, unzip it
zip_file_path = "/content/parquet_folder.zip"
extract_to = "./"

shutil.unpack_archive(zip_file_path, extract_to)
print("Extraction completed!")


Extraction completed!


In [4]:
import pandas as pd
import glob

# Find all parquet files in the extracted folder
parquet_files = glob.glob(f"{extract_to}/**/*.parquet", recursive=True)

# Read and concatenate all the Parquet files into a single DataFrame
parquet_df = pd.concat([pd.read_parquet(file) for file in parquet_files], ignore_index=True)


In [5]:
# Display combined_df
parquet_df.head()

,pickup_datetime,dropoff_datetime,passenger_count,pickup_location,dropoff_location,store_and_fwd_flag,trip_duration,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,2024-01-01 17:59:56,2024-01-01 19:17:33,1.0,26,243,N,4657,169148.407469,987397.462666,251552.228689,1.002831e+06
1,2024-01-01 23:29:09,2024-01-01 23:29:12,2.0,26,264,N,3,169148.407469,987397.462666,NaN,NaN
2,2024-01-02 06:51:53,2024-01-02 07:24:28,1.0,26,29,N,1955,169148.407469,987397.462666,150924.276878,9.950231e+05
3,2024-01-02 06:46:54,2024-01-02 07:38:39,1.0,29,209,N,3105,150924.276878,995023.095008,197610.823578,9.832340e+05
4,2024-01-02 07:54:40,2024-01-02 09:08:15,1.0,29,246,N,4415,150924.276878,995023.095008,213727.400082,9.831375e+05


In [6]:
parquet_df.isna().sum()

,0
pickup_datetime,0
dropoff_datetime,0
passenger_count,140254
pickup_location,0
dropoff_location,0
store_and_fwd_flag,140254
trip_duration,0
pickup_latitude,12030
pickup_longitude,12030
dropoff_latitude,28136


In [8]:
# Check shape, verify size of table
parquet_df.shape

(2965795, 11)

In [11]:
# Drop all the NaN values
parquet_df = parquet_df.dropna()

In [12]:
# Check shape, verify size of table
parquet_df.shape

(2794810, 11)

Noticed something: a large quantity of NaN values. This was overlooked in initial stages of the process. Removed now.

Next place this DataFrame into a single parquet file.

In [13]:
# Create a single `parquet` file instead of a folder, easier for s3 during tests
parquet_df.to_parquet("NYCtaxi.parquet")
print("Combined parquet file saved as 'NYCTaxi.parquet'")


Combined parquet file saved as 'NYCTaxi.parquet'


This single file was downloaded successfully from a URL. Now let's try reading it into this code.

Many tests have not worked. For now the single file can be found [Resources](/Resources) folder. The file is zipped because of storage issues on github repo. 

Use the following code to extract the zipped file.

In [ ]:
import shutil

# Find zipped file, unzip it
zip_file_path = "/content/parquet_file.zip"
extract_to = "./"

shutil.unpack_archive(zip_file_path, extract_to)
print("Extraction completed!")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming parquet_df is your pandas-like DataFrame

# Split the DataFrame into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(parquet_df, test_size=0.2, random_state=42)

# Show the split counts (optional)
print(f"Training rows: {len(train_df)}, Testing rows: {len(test_df)}")

# Save the DataFrames to CSV files
train_df.to_csv('new_train.csv', index=False)
test_df.to_csv('new_test.csv', index=False)

print("CSV files saved successfully!")


In [ ]:
# Create a zip file the two large CSVs
train_location = "/content/new_train.csv"
test_location = "/content/new_test.csv"

shutil.make_archive('/content/new_train', 'zip', train_location)
shutil.make_archive('/content/new_test', 'zip', test_location)